# 🔄⚡ Logic Apps + Semantic Kernel Hybrid Tutorial

**🚀 Learn how to combine Azure Logic Apps with Semantic Kernel plugins for powerful workflow automation!**

This tutorial demonstrates:
1. **Logic Apps Integration** - Automated workflow triggers using AzureLogicAppTool
2. **Semantic Kernel Plugins** - Local data processing capabilities
3. **Hybrid Agent** - Combining workflows with AI intelligence

**Scenario:** An intelligent business assistant that can:
- 📧 Trigger email workflows via Logic Apps
- 📊 Process business data locally with SK plugins
- 🔗 Coordinate between cloud automation and AI analysis

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name
- `AZURE_SUBSCRIPTION_ID`: Your Azure subscription ID
- `AZURE_RESOURCE_GROUP_NAME`: Resource group containing your Logic App

**Architecture Overview:**
```
Azure AI Agent
    ├── AzureLogicAppTool → Email/Notification Workflows
    └── SK Plugins
            ├── DataPlugin → process_sales_data()
            └── ReportPlugin → generate_summary()
```

![Azure Tools](images/azure_tools.gif)

## 📦 Installing Dependencies and Imports

First, we'll install the necessary packages and import the required modules for our hybrid solution. This includes:

- **Azure AI Agents SDK** - For creating intelligent agents
- **Azure Logic Apps Management** - For managing and triggering Logic Apps
- **Semantic Kernel** - For local AI processing and plugins
- **Azure Identity** - For authentication with Azure services

The following cell sets up all the dependencies we need for this tutorial.

In [1]:
# Install required packages
# !pip install azure-ai-agents azure-identity azure-mgmt-logic requests

import os
import json
import requests
from typing import Annotated, Dict, Any, Callable, Set
from datetime import datetime, timedelta

# Azure AI Agents imports
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential

# Azure Logic Apps Management
from azure.mgmt.logic import LogicManagementClient

# Semantic Kernel imports
import semantic_kernel as sk
from semantic_kernel.functions import kernel_function

print("✅ Required packages imported successfully!")

✅ Required packages imported successfully!


## 🔧 Building the Azure Logic Apps Tool

Now we'll implement the core `AzureLogicAppTool` class that serves as our bridge between the AI agent and Azure Logic Apps. This tool:

1. **Registers Logic Apps** - Retrieves callback URLs for Logic App triggers
2. **Manages Multiple Workflows** - Stores and organizes multiple Logic App endpoints
3. **Handles Invocation** - Sends HTTP requests to trigger Logic App workflows
4. **Provides Fallback** - Simulates actions when Logic Apps aren't available

This implementation allows our AI agent to seamlessly trigger cloud-based automation workflows.

In [12]:
# Azure Logic Apps Tool Implementation
class AzureLogicAppTool:
    """
    A service that manages multiple Logic Apps by retrieving and storing their callback URLs,
    and then invoking them with an appropriate payload.
    """

    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.logic_client = LogicManagementClient(credential, subscription_id)
        self.callback_urls: Dict[str, str] = {}

    def register_logic_app(self, logic_app_name: str, trigger_name: str) -> None:
        """
        Retrieves and stores a callback URL for a specific Logic App + trigger.
        Raises a ValueError if the callback URL is missing.
        """
        try:
            callback = self.logic_client.workflow_triggers.list_callback_url(
                resource_group_name=self.resource_group,
                workflow_name=logic_app_name,
                trigger_name=trigger_name,
            )

            if callback.value is None:
                raise ValueError(f"No callback URL returned for Logic App '{logic_app_name}'.")

            self.callback_urls[logic_app_name] = callback.value
            print(f"✅ Registered Logic App: {logic_app_name}")
        except Exception as e:
            print(f"⚠️ Warning: Could not register Logic App '{logic_app_name}': {e}")
            print("💡 This is expected if the Logic App doesn't exist - emails will be simulated")

    def invoke_logic_app(self, logic_app_name: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        """
        Invokes the registered Logic App (by name) with the given JSON payload.
        Returns a dictionary summarizing success/failure.
        """
        if logic_app_name not in self.callback_urls:
            # Simulate email sending if Logic App not registered
            print(f"📧 SIMULATED EMAIL:")
            print(f"   To: {payload.get('to', 'unknown')}")
            print(f"   Subject: {payload.get('subject', 'unknown')}")
            print(f"   Body: {payload.get('body', 'unknown')}")
            return {"result": f"Successfully simulated email via {logic_app_name}."}

        url = self.callback_urls[logic_app_name]
        try:
            response = requests.post(url=url, json=payload, timeout=30)
            if response.ok:
                print(f"✅ Successfully invoked Logic App: {logic_app_name}")
                return {"result": f"Successfully invoked {logic_app_name}."}
            else:
                print(f"⚠️ Error invoking Logic App: {logic_app_name} ({response.status_code})")
                return {"error": f"Error invoking {logic_app_name} ({response.status_code}): {response.text}"}
        except Exception as e:
            print(f"⚠️ Exception invoking Logic App: {logic_app_name} - {str(e)}")
            return {"error": f"Error invoking {logic_app_name}: {str(e)}"}

def create_send_email_function(service: AzureLogicAppTool, logic_app_name: str) -> Callable[[str, str, str], str]:
    """
    Returns a function that sends an email by invoking the specified Logic App.
    """
    def send_email_via_logic_app(to: str, subject: str, body: str) -> str:
        """
        Sends an email by invoking the specified Logic App with the given recipient, subject, and body.
        """
        payload = {
            "to": to,
            "subject": subject,
            "body": body,
        }
        result = service.invoke_logic_app(logic_app_name, payload)
        print("📧 Email sent via Logic App:")
        return json.dumps(result)

    return send_email_via_logic_app

print("🔧 AzureLogicAppTool class defined!")

🔧 AzureLogicAppTool class defined!


## 📧 Creating Agent with Azure Logic Apps Integration

We'll create an AI agent that can trigger Logic Apps workflows for automated business processes.

## 🎯 Initializing Azure AI Agents Client

Before we can create our hybrid agent, we need to establish a connection to Azure AI services. This step:

- **Connects to Azure AI Project** - Uses your project endpoint for agent creation
- **Sets Up Authentication** - Uses DefaultAzureCredential for secure access
- **Configures Model Deployment** - Specifies which AI model to use for the agent

Make sure you have set the required environment variables in your development environment.

In [3]:
# Initialize Azure AI Agents client
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP_NAME")

if not all([project_endpoint, model_deployment, subscription_id, resource_group]):
    raise ValueError("Missing required environment variables. Please set PROJECT_ENDPOINT, MODEL_DEPLOYMENT_NAME, AZURE_SUBSCRIPTION_ID, and AZURE_RESOURCE_GROUP_NAME")

credential = DefaultAzureCredential()
agents_client = AgentsClient(endpoint=project_endpoint, credential=credential)

print(f"✅ Connected to Azure AI Project")
print(f"📊 Using model: {model_deployment}")

✅ Connected to Azure AI Project
📊 Using model: gpt-4.1


## 🔗 Registering Logic Apps and Creating Email Function

In this step, we configure our Logic App integration by:

1. **Registering the Logic App** - Connecting to the specific Logic App workflow
2. **Retrieving Callback URLs** - Getting the HTTP trigger endpoint
3. **Creating Email Function** - Building a specialized function for the agent to use

The Logic App should have an HTTP trigger named "When_a_HTTP_request_is_received" that can accept email parameters (to, subject, body).

In [13]:
# Logic App details
logic_app_name = "agent-logic-apps"
trigger_name = "When_a_HTTP_request_is_received"

# Create and initialize AzureLogicAppTool utility
logic_app_tool = AzureLogicAppTool(subscription_id, resource_group)
logic_app_tool.register_logic_app(logic_app_name, trigger_name)
print(f"Registered logic app '{logic_app_name}' with trigger '{trigger_name}'.")

# Create the specialized "send_email_via_logic_app" function for your agent tools
send_email_func = create_send_email_function(logic_app_tool, logic_app_name)

print(f"✅ Logic App tool configured for: {logic_app_name}")
print(f"🔧 Trigger: {trigger_name}")
print(f"📧 Email function created for workflow integration")

⚠️ Warning: Could not register Logic App 'agent-logic-apps': (ResourceNotFound) The Resource 'Microsoft.Logic/workflows/agent-logic-apps' under resource group 'azure-ai-agents-playbook' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Logic/workflows/agent-logic-apps' under resource group 'azure-ai-agents-playbook' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
💡 This is expected if the Logic App doesn't exist - emails will be simulated
Registered logic app 'agent-logic-apps' with trigger 'When_a_HTTP_request_is_received'.
✅ Logic App tool configured for: agent-logic-apps
🔧 Trigger: When_a_HTTP_request_is_received
📧 Email function created for workflow integration


In [16]:
logic_app_name

'agent-logic-apps'

## 🤖 Creating the Hybrid Business Agent

Now we'll create our intelligent business agent with Logic Apps integration. This process involves:

1. **Function Registration** - Adding our email function to the agent's toolset
2. **Tool Configuration** - Enabling auto function calls for seamless execution
3. **Agent Instructions** - Defining the agent's role and capabilities
4. **Workflow Integration** - Connecting AI intelligence with automated processes

The agent will be able to understand business requests and automatically trigger appropriate Logic App workflows.

In [ ]:
# Prepare the function tools for the agent
functions_to_use: Set = {
    send_email_func,  # This references the AzureLogicAppTool instance via closure
}

# Create the Logic Apps agent with proper toolset
# Create function tool and toolset
functions = FunctionTool(functions=functions_to_use)
toolset = ToolSet()
toolset.add(functions)

# Enable auto function calls - this is key for actual execution
agents_client.enable_auto_function_calls(toolset)

logic_apps_agent = agents_client.create_agent(
    model=model_deployment,
    name="LogicAppsBusinessAgent",
    instructions="""
    You are a business automation assistant that can trigger workflow processes.
    
    Your capabilities:
    - Send email notifications via Logic Apps using send_email_via_logic_app function
    - Trigger alerts for business events
    - Coordinate automated workflows
    
    When sending emails, use the send_email_via_logic_app function with:
    - to: email address
    - subject: email subject line
    - body: email content
    
    Always explain what workflow you're triggering and why.
    """,
    toolset=toolset
)

print(f"🤖 Logic Apps Agent created: {logic_apps_agent.id}")
print(f"🔧 Agent configured with email workflow capabilities")

## 🧪 Testing the Logic Apps Integration

Time to test our hybrid agent! In this section we will:

1. **Configure Email Address** - Set your email for testing notifications
2. **Create Thread** - Start a conversation with the agent
3. **Send Test Message** - Request a business workflow trigger
4. **Monitor Execution** - Watch as the agent processes and executes the Logic App

The agent should intelligently understand the request and automatically trigger the appropriate email workflow.

#### Input Your Email Address



In [ ]:
# please replace with your email address
my_email = "test@test.com"

## 🎬 Executing the Business Workflow

Watch the magic happen! This cell demonstrates the complete flow:

```mermaid
graph TD
    A[User Request] --> B[AI Agent Processing]
    B --> C{Logic Apps Available?}
    C -->|Yes| D[Trigger Logic App]
    C -->|No| E[Simulate Email]
    D --> F[Send Real Email]
    E --> G[Display Simulated Email]
    F --> H[Return Success]
    G --> H
```

The agent will:
- Parse the business request
- Identify the need for email notification
- Execute the `send_email_via_logic_app` function
- Trigger the Logic App workflow (or simulate if unavailable)

In [ ]:
# Test the Logic Apps agent
thread = agents_client.threads.create()

# Send a test message
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Send a notification email about high sales performance this quarter to this email address {my_email}.",
)

# Create and process an agent run in the thread
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=logic_apps_agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Get the response
print("📧 Logic Apps Agent Response:")
messages = agents_client.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.text_messages:
        print(f"\n🗨️ {msg.role.upper()}: {msg.text_messages[-1].text.value}")
        print("-" * 80)




## 🎉 Conclusion

**You've successfully created a hybrid Azure AI Agent that combines:**

### 🔄 Azure Logic Apps Integration
- Used `AzureLogicAppTool` for workflow automation
- Registered Logic Apps with `register_logic_app()` method
- Triggered email notifications and business processes

### 🔌 Semantic Kernel Plugins
- Created local data processing capabilities
- Built custom business intelligence functions
- Processed sales data and generated reports

### 🎯 Hybrid Orchestration
- Combined cloud automation with local AI processing
- Coordinated between multiple tools and workflows
- Created intelligent business automation

**Next Steps:**
- Expand SK plugins for more business logic
- Add more Logic Apps for different workflows
- Implement error handling and monitoring
- Scale to enterprise-level scenarios

---
*Happy automating! 🚀*